In [2]:
setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/bulk/hahn_2023/cortex")

source("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/code/enrichment_fxns.R")

source("/mnt/lareaulab/reliscu/code/upper_first.R")

Here I perform enrichment analysis to find modules enriched for cell type markers. 

These modules will later be used to correlate with exon PSI to define cell type-specific exons.

In [3]:
mod_def <- "PosBC"

# Prep marker genes

## Yao et. al 2021 marker genes

In [18]:
# # Use marker genes from Yao et. al 2021:

# marker_genes_df <- fread("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/scRNA-seq/yao_2021/yao_2021_tableS6_marker_genes.txt", data.table=FALSE)

# head(marker_genes_df)

,V1,cl1_cluster_label,cl2_cluster_label,cl1_cluster_id,cl2_cluster_id,direction,genes
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,1,Lamp5 Pax6,Lamp5 Egln3_1,1,2,up,Nxph1
2,2,Lamp5 Pax6,Lamp5 Egln3_1,1,2,up,Luzp2
3,3,Lamp5 Pax6,Lamp5 Egln3_1,1,2,up,Scrg1
4,4,Lamp5 Pax6,Lamp5 Egln3_1,1,2,up,Pax6
5,5,Lamp5 Pax6,Lamp5 Egln3_1,1,2,up,Ano3
6,6,Lamp5 Pax6,Lamp5 Egln3_1,1,2,up,Pbx3


In [ ]:
# ctypes <- unique(marker_genes_df$cl1_cluster_label)

# marker_genes_list <- lapply(ctypes, function(w_ctype) {
#     working_ctype_tests <- marker_genes_df[(marker_genes_df$cl1_cluster_label %in% w_ctype),]
#     de_genes_per_test <- lapply(ctypes[ctypes != w_ctype], function(o_ctype) {
#         if (o_ctype != w_ctype) {
#             working_ctype_tests$gene[(working_ctype_tests$cl2_cluster_label %in% o_ctype) & (working_ctype_tests$direction %in% "up")]
#         }
#     })
#     hits_per_gene <- table(unlist(de_genes_per_test))
#     names(hits_per_gene)[hits_per_gene > 60] 
# })
# names(marker_genes_list) <- ctypes

In [ ]:
# marker_genes_list <- marker_genes_list[lengths(marker_genes_list) > 1]

## Pseudobulk DE genes

In [27]:
unique <- FALSE

In [28]:
# pairwise_res_list <- readRDS("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM/data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")
# pairwise_ctype_genes <- prep_DE_genes(pairwise_res_list, lfc_threshold, pairwise=TRUE, unique=unique)

pairwise_res_list <- readRDS("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/yao_2021/ACA/data/DE_genes/yao_2021_ACA_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")
pairwise_ctype_genes <- prep_DE_genes(pairwise_res_list, lfc_threshold, pairwise=TRUE, unique=unique)

marker_genes_list <- pairwise_ctype_genes

## MO marker genes

In [12]:
load("/mnt/lareaulab/reliscu/data/gene_sets/Oldham/MO_sets.RData")

In [98]:
# MO_legend[grepl("GLUTA", MO_legend$SetName) & grepl("CellType", MO_legend$Category),]

In [13]:
sets <- c(
    "BARRES_ASTROCYTES",
    "BARRES_OLIGODENDROCYTES",
    "HICKMAN_MICROGLIA_SENSOME_99",
    "BARRES_NEURONS",
    "BUTLER_ENDOTHELIAL_HPA",
    "ZEISEL_MURAL",
    "ABA_CHOROID_PLEXUS",
    "ZEISEL_EPENDYMAL",
    "ZHANG_MOUSE_OPC_2014",
    "ZEISEL_INTERNEURON",
    "SUGINO_UP_GABAERGIC_NEURONS",
    "SUGINO_UP_GLUTAMATERGIC_NEURONS"
)

In [19]:
mask <- MO_legend$SetName %in% sets

marker_genes_list <- MO_sets[mask]
names(marker_genes_list) <- MO_legend$SetName[mask] 

# Format genes to match mouse symbols

marker_genes_list <- lapply(marker_genes_list, function(x) unname(sapply(x, upper_first)))

# Get enrichments

In [29]:
set_source <- "yao_ACA_pseudobulk"

In [30]:
network_dir <- "hahn_2023_cortex_STAR_TPM_SN_QN_mergeParam0.85_Modules"

In [31]:
enrichments_df <- get_module_enrichments(network_dir, marker_genes_list, mod_def)

In [32]:
# Get most enriched cell type for each module
# If cell type is most enriched in multiple modules, choose module with smallest p-value

top_mods_df <- enrichments_df %>%
    group_by(Cell_type) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    group_by(Network, Module) %>%
    arrange(Network) %>%
    slice_min(Qval, with_ties=FALSE) %>%
    arrange(Qval)

In [23]:
top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
BARRES_OLIGODENDROCYTES,1.032214e-83,2.092092e-79,brown,Bicor-None_signum0.379_minSize10_merge_ME_0.85_20371
BARRES_ASTROCYTES,3.483482e-29,2.277523e-26,green,Bicor-None_signum0.627_minSize8_merge_ME_0.85_20371
SUGINO_UP_GLUTAMATERGIC_NEURONS,4.523060e-25,2.777981e-22,turquoise,Bicor-None_signum0.349_minSize15_merge_ME_0.85_20371
HICKMAN_MICROGLIA_SENSOME_99,1.033682e-22,5.371966e-20,salmon1,Bicor-None_signum0.196_minSize8_merge_ME_0.85_20371
ABA_CHOROID_PLEXUS,4.125885e-22,1.944731e-19,yellow,Bicor-None_signum0.379_minSize12_merge_ME_0.85_20371
ZEISEL_INTERNEURON,1.093184e-21,4.431332e-19,blue,Bicor-None_signum0.196_minSize10_merge_ME_0.85_20371
BARRES_NEURONS,4.296677e-20,1.000978e-17,blue,Bicor-None_signum0.627_minSize15_merge_ME_0.85_20371
ZEISEL_EPENDYMAL,9.939308e-19,1.798660e-16,brown,Bicor-None_signum0.379_minSize15_merge_ME_0.85_20371
SUGINO_UP_GABAERGIC_NEURONS,1.225537e-18,2.198158e-16,blue,Bicor-None_signum0.349_minSize15_merge_ME_0.85_20371


In [33]:
top_mods_df[,c("Cell_type", "Pval", "Qval", "Module", "Network")]

Cell_type,Pval,Qval,Module,Network
<chr>,<dbl>,<dbl>,<chr>,<chr>
oligodendrocyte,1.127345e-91,2.856128e-87,yellow,Bicor-None_signum0.379_minSize15_merge_ME_0.85_20371
L4_5_intratelencephalic_projecting_glutamatergic_neuron_of_the_primary_motor_cortex,2.639658e-38,2.089867e-35,turquoise,Bicor-None_signum0.379_minSize12_merge_ME_0.85_20371
L6_corticothalamic-projecting_glutamatergic_cortical_neuron,2.774355e-26,1.191327e-23,brown,Bicor-None_signum0.476_minSize15_merge_ME_0.85_20371
pvalb_GABAergic_cortical_interneuron,4.535552e-22,1.689826e-19,tan,Bicor-None_signum0.476_minSize12_merge_ME_0.85_20371
endothelial_cell,1.961191e-18,6.289466e-16,red,Bicor-None_signum0.627_minSize6_merge_ME_0.85_20371
L5_6_near-projecting_glutamatergic_neuron_of_the_primary_motor_cortex,1.226043e-12,2.610236e-10,blue,Bicor-None_signum0.476_minSize12_merge_ME_0.85_20371
sst_chodl_GABAergic_cortical_interneuron,3.956480e-12,8.018994e-10,yellow,Bicor-None_signum0.627_minSize6_merge_ME_0.85_20371
L6b_glutamatergic_cortical_neuron,5.295223e-12,1.064718e-09,orangered4,Bicor-None_signum0.476_minSize6_merge_ME_0.85_20371
L6_intratelencephalic_projecting_glutamatergic_neuron_of_the_primary_motor_cortex,2.014096e-09,2.758223e-07,blue,Bicor-None_signum0.196_minSize12_merge_ME_0.85_20371


In [34]:
write.csv(top_mods_df, file=paste0("data/enrichments/hahn_2023_cortex_STAR_TPM_top_", set_source, "_Qval_modules.csv"), row.names=FALSE, quote=FALSE)